<center><img src="../Picture Data/logo.png" alt="Header" style="width: 800px;"/></center>

@Copyright (C): 2010-2019, Shenzhen Yahboom Tech  
@Author: Malloy.Yuan  
@Date: 2019-07-17 10:10:02  
@LastEditors: Malloy.Yuan  
@LastEditTime: 2019-09-17 17:54:19  

# Load training model
After running the 'training model' to train the data collected by the 'Data collection' code into the obstacle avoidance model we need, we now use this model step by step to achieve obstacle avoidance.

Execute the following code to initialize the PyTorch model

In [ ]:
import torch
import torchvision

model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

Load your own uploaded, ``best_model.pth`` model that has been trained

In [ ]:
model.load_state_dict(torch.load('best_model.pth'))

At present, the model weight calculation is located in the CPU memory. We transfer the model to the GPU through the introduction of our previous tutorial or 'CUDA', and execute the following code to use the GPU.

In [ ]:
device = torch.device('cuda')
model = model.to(device)

# pre-processing function
We need to do some pre-processing to ensure that the image format of our camera is exactly the same as the image format when training the model.
Need to perform the following steps:
1. Convert from BGR to RGB mode
2. Convert from HWC layout to CHW layout
3. Normalize using the same parameters as during training (our camera provides values in the range [0, 255] and trains the loaded image in the range [0, 1], so we need to scale 255.0)
4. Transfer data from CPU memory to GPU memory
5. Add dimensions in bulk

In [ ]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

Once the preprocessing function is defined, the image can be converted from a camera format to a neural network input format.

We also need to create a slider that shows the probability that the robot is blocked,
Code as shown below:

In [ ]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image, blocked_slider]))

Create a robot instance of the drive motor.

In [ ]:
from jetbot import Robot

robot = Robot()

Then,we need to create a function that calls the function whenever the value of the camera changes. This feature will perform the following steps:
1. Pre-process camera image
2. Perform a neural network
3. When the neural network output indicates that we are blocked, we will turn left, otherwise we will move on.
Code as shown below:

In [ ]:
import torch.nn.functional as F
import time

def update(change):
    global blocked_slider, robot
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    prob_blocked = float(y.flatten()[0])
    
    blocked_slider.value = prob_blocked
    
    #if prob_blocked < 0.5:
        #robot.forward(0.4)
    #else:
        #robot.left(0.4)
    if prob_blocked < 0.78:
        robot.stop()
#         robot.forward(0.7)
    else:
        robot.stop()
#         robot.left(0.7)
    
    time.sleep(0.001)
        
update({'new': camera.value})  # we call the function once to intialize

We have created a neural network execution function, but now we need to attach it to the camera for processing. We do this with the ``observe`` function.

Tips: The robot will start moving at this time! Make sure your Jetbot robot is in a movable area to avoid falling damage!!!

In [ ]:
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

After running the above code block, Jetbot starts generating new commands for each detected picture.

First, we need to put the robot on the ground and observe its reaction when it encounters an obstacle.

If you want to stop performing autonomous obstacle avoidance, you can cancel by executing the following code.

In [ ]:
camera.unobserve(update, names='value')
time.sleep(1)
robot.stop()

Execute the code shown below, you can cancel the camera connection, just do not push the video data stream to the browser, but the Jetbot camera is still working.

In [ ]:
camera_link.unlink()  # don't stream to browser (will still run camera)

If you need to re-display refresh, run the following code.

In [ ]:
camera_link.link()  # stream to browser (wont run camera)